In [1]:
# 구글드라이브에 업로드한 데이터를 Colab에서 사용하기 위해서 Mount를 수행합니다.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 수집한 데이터들중 원하는 가져오기 위해 키워드를 입력합니다.
query_keyword = input("분석을 위한 키워드를 입력하세요 : ")

분석을 위한 키워드를 입력하세요 : 경제


In [ ]:
import pickle
base_path = "/content/drive/MyDrive/Colab Notebooks/dataset/"

with open(base_path + f"tokenized_docs({query_keyword}).pk", "rb") as f:
    tokenized_docs = pickle.load(f)

In [ ]:
# 텍스트의 단어를 셀 수 있는 Counter 함수를 불러옵니다.
from collections import Counter

total_tokens = [token for doc in tokenized_docs for token in doc]
print("Number of Total tokens : ", len(set(total_tokens)))

# 각 token 별로 빈도를 계산해주는 Counter 객체.
token_counter = Counter(total_tokens)

# 빈도 높은 50개의 token을 출력합니다. 빈도 top50에 대해서는 token과 빈도를 wordInfo라는 사전에 저장합니다.
wordInfo = dict()
print("\n--Token : Freq--")
for tags, counts in token_counter.most_common(50): # top 50개 출력.
    wordInfo[tags] = counts
    print("%6s : %d" % (tags, counts))

In [ ]:
# 그래프를 이쁘게 그리기 위한 코드입니다. 한글 글꼴을 추가합니다.

%matplotlib inline  

import matplotlib as mpl  # 기본 설정 만지는 용도
import matplotlib.pyplot as plt  # 그래프 그리는 용도
import matplotlib.font_manager as fm  # 폰트 관련 용도
import seaborn as sns
mpl.rcParams['axes.unicode_minus'] = False

sys_font=fm.findSystemFonts()
print(f"sys_font number: {len(sys_font)}")
print(sys_font)

nanum_font = [f for f in sys_font if 'Nanum' in f]
print(f"nanum_font number: {len(nanum_font)}")

!apt-get update -qq
!apt-get install fonts-nanum* -qq

path = '/usr/share/fonts/truetype/nanum/NanumBarunGothicBold.ttf'  # 설치된 나눔글꼴중 원하는 녀석의 전체 경로를 가져옵니다.
font_name = fm.FontProperties(fname=path, size=10).get_name()
print(font_name)
plt.rc('font', family=font_name)

In [ ]:
# 현재 설정되어 있는 폰트 사이즈와 글꼴을 알아보자
!python --version
def current_font():
  print(f"설정 폰트 글꼴: {plt.rcParams['font.family']}, 설정 폰트 사이즈: {plt.rcParams['font.size']}")  # 파이썬 3.6 이상 사용가능하다
        
current_font()

In [ ]:
# 여전히 글꼴이 보이지 않는 분들은, 런타임 -> "다시 시작 및 모두 실행" 을 눌러주세요!
fm._rebuild()

In [ ]:
from wordcloud import WordCloud

# 설정된 font(나눔고딕)를 기준으로, Wordcloud를 출력합니다. 색상은 twilight theme를 사용하고, 배경은 흰색으로 합니다.
wordcloud = WordCloud(font_path = path,
                      relative_scaling = 0.2,
                      background_color='white',
                      colormap="twilight"
                      ).generate_from_frequencies(wordInfo)
plt.figure(figsize=(16,16))
plt.imshow(wordcloud)
plt.axis("off")
plt.savefig(base_path + f"wordcloud_{query_keyword}.png")
plt.show()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

#TfidfVectorizer의 input으로 만들기 위한 전처리.
tfidf_docs = [" ".join(doc) for doc in tokenized_docs]

tfidf = TfidfVectorizer()

# tfidf 형태로 변환.
X_tfidf = tfidf.fit_transform(tfidf_docs)

terms = tfidf.get_feature_names()

# sum tfidf frequency of each term through documents
sums = X_tfidf.sum(axis=0)

# connecting term to its sums frequency
df = []
for col, term in enumerate(terms):
    df.append( (term, sums[0,col] ))

ranking = pd.DataFrame(df, columns=['Term','TF-IDF'])
rankInfo = ranking.sort_values('TF-IDF', ascending=False)[:50]

x = list(rankInfo["Term"])
y = list(rankInfo["TF-IDF"])

tfidfInfo = dict()

for vocab, tfidf in zip(x, y):
    tfidfInfo[vocab] = tfidf

print(x[:5])
print(y[:5])
print()
print(rankInfo)

In [ ]:
from wordcloud import WordCloud

# 설정된 font(나눔고딕)를 기준으로, Wordcloud를 출력합니다. 색상은 twilight theme를 사용하고, 배경은 흰색으로 합니다.
wordcloud = WordCloud(font_path = path,
                      relative_scaling = 0.2,
                      background_color='white',
                      colormap="twilight"
                      ).generate_from_frequencies(tfidfInfo)
plt.figure(figsize=(16,16))
plt.imshow(wordcloud)
plt.axis("off")
plt.savefig(base_path + f"wordcloud_{query_keyword}(TFIDF).png")
plt.show()